In [8]:

import datetime as dt
import requests
import pandas as pd
import numpy as np
import matplotlib
import mplfinance as mpf
import talib

In [9]:

def now(lookback): 
    #get time now convert it to unix
    timenow = dt.datetime.utcnow()
    end_time = timenow.replace(tzinfo=dt.timezone.utc).timestamp()
    #get lookback and convert to unix 
    start_time = dt.datetime.utcnow() - dt.timedelta(lookback)
    start_time = start_time.replace(tzinfo=dt.timezone.utc).timestamp() 
    return start_time, end_time

_start_time, _end_time = now(3)

data = requests.get(f"https://ftx.com/api//markets/{'BTC-PERP'}/candles?resolution={3600}&start_time={_start_time}&end_time={_end_time}").json()

data = pd.DataFrame(data['result'])

data['time'] = pd.to_datetime(data['time'], unit='ms')
data.set_index('time', inplace= True)
#print(data.head(5))

In [10]:
def getmarkets(PERP):
    markets = requests.get(f"https://ftx.com/api/markets").json()
    markets = pd.DataFrame(markets['result'])
    markets = markets['name'].tolist()
    markets = [market for market in markets if market.endswith(PERP)]
    return markets

perpnames = getmarkets('PERP')
print(perpnames)

['1INCH-PERP', 'AAVE-PERP', 'ADA-PERP', 'AGLD-PERP', 'ALCX-PERP', 'ALGO-PERP', 'ALICE-PERP', 'ALPHA-PERP', 'ALT-PERP', 'AMPL-PERP', 'AR-PERP', 'ASD-PERP', 'ATLAS-PERP', 'ATOM-PERP', 'AUDIO-PERP', 'AVAX-PERP', 'AXS-PERP', 'BADGER-PERP', 'BAL-PERP', 'BAND-PERP', 'BAO-PERP', 'BAT-PERP', 'BCH-PERP', 'BIT-PERP', 'BNB-PERP', 'BNT-PERP', 'BOBA-PERP', 'BRZ-PERP', 'BSV-PERP', 'BTC-PERP', 'BTT-PERP', 'C98-PERP', 'CAKE-PERP', 'CEL-PERP', 'CELO-PERP', 'CHR-PERP', 'CHZ-PERP', 'CLV-PERP', 'COMP-PERP', 'CONV-PERP', 'CREAM-PERP', 'CRO-PERP', 'CRV-PERP', 'CUSDT-PERP', 'CVC-PERP', 'DASH-PERP', 'DAWN-PERP', 'DEFI-PERP', 'DENT-PERP', 'DODO-PERP', 'DOGE-PERP', 'DOT-PERP', 'DRGN-PERP', 'DYDX-PERP', 'EDEN-PERP', 'EGLD-PERP', 'ENJ-PERP', 'ENS-PERP', 'EOS-PERP', 'ETC-PERP', 'ETH-PERP', 'EXCH-PERP', 'FIDA-PERP', 'FIL-PERP', 'FLM-PERP', 'FLOW-PERP', 'FTM-PERP', 'FTT-PERP', 'GALA-PERP', 'GRT-PERP', 'HBAR-PERP', 'HNT-PERP', 'HOLY-PERP', 'HOT-PERP', 'HT-PERP', 'HUM-PERP', 'ICP-PERP', 'ICX-PERP', 'IMX-PERP', 'IOTA-P

In [11]:
def ohlc(perpnames, tf, daysback):
    _start_time, _end_time = now(daysback) 
    data = requests.get(f"https://ftx.com/api//markets/{perpnames}/candles?resolution={tf}&start_time={_start_time}&end_time={_end_time}").json()
    data = pd.DataFrame(data['result'])
    return data

In [12]:
dfs = []
tf = 300
daysback = 2
for perp in perpnames: 
    dfs.append(ohlc(perp, tf, daysback))

In [13]:
all = pd.concat(dict(zip(perpnames,dfs)), axis=1)
#closes = all.loc[:,all.columns.get_level_values(1).isin(['close'])]

nameless = pd.concat(dfs, axis=1)
closes = nameless.loc[:,nameless.columns.get_level_values(0).isin(['close'])]
all

1INCH-PERP                                                \
                     startTime          time    open    high     low   close   
0    2022-03-12T15:50:00+00:00  1.647100e+12  1.3557  1.3561  1.3502  1.3508   
1    2022-03-12T15:55:00+00:00  1.647100e+12  1.3508  1.3555  1.3492  1.3521   
2    2022-03-12T16:00:00+00:00  1.647101e+12  1.3521  1.3543  1.3517  1.3527   
3    2022-03-12T16:05:00+00:00  1.647101e+12  1.3527  1.3563  1.3527  1.3548   
4    2022-03-12T16:10:00+00:00  1.647101e+12  1.3548  1.3594  1.3531  1.3579   
..                         ...           ...     ...     ...     ...     ...   
571  2022-03-14T15:25:00+00:00  1.647272e+12  1.3308  1.3315  1.3296  1.3306   
572  2022-03-14T15:30:00+00:00  1.647272e+12  1.3306  1.3313  1.3282  1.3293   
573  2022-03-14T15:35:00+00:00  1.647272e+12  1.3293  1.3294  1.3267  1.3277   
574  2022-03-14T15:40:00+00:00  1.647272e+12  1.3277  1.3283  1.3254  1.3257   
575  2022-03-14T15:45:00+00:00  1.647273e+12  1.3257  1.3263  1.3240  1.3252   

                                 AAVE-PERP                        ...  \
         volume                  startTime          time    open  ...   
0    34431.7835  2022-03-12T15:50:00+00:00  1.647100e+12  119.16  ...   
1    32424.4324  2022-03-12T15:55:00+00:00  1.647100e+12  118.93  ...   
2    17450.1974  2022-03-12T16:00:00+00:00  1.647101e+12  118.90  ...   
3    35402.2321  2022-03-12T16:05:00+00:00  1.647101e+12  118.67  ...   
4    35019.0949  2022-03-12T16:10:00+00:00  1.647101e+12  118.81  ...   
..          ...                        ...           ...     ...  ...   
571  23441.2334  2022-03-14T15:25:00+00:00  1.647272e+12  117.91  ...   
572  12638.0826  2022-03-14T15:30:00+00:00  1.647272e+12  117.85  ...   
573  20514.4396  2022-03-14T15:35:00+00:00  1.647272e+12  117.61  ...   
574  18514.2139  2022-03-14T15:40:00+00:00  1.647272e+12  117.45  ...   
575  16541.3202  2022-03-14T15:45:00+00:00  1.647273e+12  117.32  ...   

    ZIL-PERP                                        ZRX-PERP                \
         low    close      volume                  startTime          time   
0    0.04092  0.04097  10030.4418  2022-03-12T15:55:00+00:00  1.647100e+12   
1    0.04079  0.04080   5905.4079  2022-03-12T16:00:00+00:00  1.647101e+12   
2    0.04080  0.04094   9946.3159  2022-03-12T16:05:00+00:00  1.647101e+12   
3    0.04089  0.04099   1400.1155  2022-03-12T16:10:00+00:00  1.647101e+12   
4    0.04099  0.04107   6054.1949  2022-03-12T16:15:00+00:00  1.647102e+12   
..       ...      ...         ...                        ...           ...   
571  0.03904  0.03908   6621.5821  2022-03-14T15:30:00+00:00  1.647272e+12   
572  0.03899  0.03901   4970.8280  2022-03-14T15:35:00+00:00  1.647272e+12   
573  0.03900  0.03901   2201.7859  2022-03-14T15:40:00+00:00  1.647272e+12   
574  0.03892  0.03892  26316.0880  2022-03-14T15:45:00+00:00  1.647273e+12   
575  0.03892  0.03898   1168.1398  2022-03-14T15:50:00+00:00  1.647273e+12   

                                                     
        open     high      low    close      volume  
0    0.49800  0.49825  0.49750  0.49750  4565.02175  
1    0.49750  0.49800  0.49625  0.49650  4494.78300  
2    0.49650  0.49700  0.49625  0.49650  1538.08675  
3    0.49650  0.49700  0.49600  0.49650  4096.59600  
4    0.49650  0.49650  0.49600  0.49650   795.46650  
..       ...      ...      ...      ...         ...  
571  0.48525  0.48525  0.48450  0.48450  1830.49650  
572  0.48450  0.48500  0.48375  0.48375  1381.86450  
573  0.48375  0.48400  0.48300  0.48300   967.92775  
574  0.48300  0.48325  0.48225  0.48250  2458.41850  
575  0.48250  0.48325  0.48250  0.48325     0.48325  

[576 rows x 1253 columns]

In [14]:
def divs(closes, columns, ob=70, os=30, period=14):
    """Calculates bullish and bearish RSI divergences under oversold or overbought conditions"""

    closes['RSI'] = talib.RSI(closes.iloc[:, columns])
    closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
    closes['rolling_rsi_low'] = closes['RSI'].rolling(period).min()
    closes['rolling_closing_high'] = closes.iloc[:, columns].rolling(period).max()
    closes['rolling_closing_low'] = closes.iloc[:, columns].rolling(period).min()


    closes['new_RSI_high'] = np.where(closes['rolling_rsi_high'] > closes['rolling_rsi_high'].shift(), 1, 0)
    closes['new_RSI_low'] = np.where(closes['rolling_rsi_low'] < closes['rolling_rsi_low'].shift(), 1, 0)


    closes['new_closing_high'] = np.where(closes['rolling_closing_high'] > closes['rolling_closing_high'].shift(), 1, 0)
    closes['new_closing_low'] = np.where(closes['rolling_closing_low'] < closes['rolling_closing_low'].shift(), 1, 0)

    bearishrsidiv = np.where((closes['new_closing_high'] == 1) & (closes['new_RSI_high'] == 0) & (closes['RSI'] > ob), 1, 0)
    bullishrsidiv = np.where((closes['new_closing_low'] == 1) & (closes['new_RSI_low'] == 0) & (closes['RSI'] < os), 1, 0)
   # closes.insert(1, 'bearish_rsi_div', bearishrsidiv)
   # closes.insert(2, 'bullish_rsi_div', bullishrsidiv)
    closes['bearish_rsi_div'] = np.where((closes['new_closing_high'] == 1) & (closes['new_RSI_high'] == 0) & (closes['RSI'] > ob), 1, 0)
    closes['bullish_rsi_div'] = np.where((closes['new_closing_low'] == 1) & (closes['new_RSI_low'] == 0) & (closes['RSI'] < os), 1, 0)
    closes = closes.dropna()
    return closes[['bullish_rsi_div', 'bearish_rsi_div']]


In [15]:
closes.iloc[-1:,]


,close,close,close,close,close,close,close,close,close,close,...,close,close,close,close,close,close,close,close,close,close
575,1.3252,117.28,0.79302,0.937,97.0,0.6972,5.5275,0.2575,3082.4,0.9554,...,0.0933,0.177775,181.27,0.759975,2.9374,18535.0,1854.5,151.99,0.03898,0.48325


In [16]:
rsidata = [] 
for column in range(179):
   rsidata.append(divs(closes, column))
alle = pd.concat(dict(zip(perpnames,rsidata)), axis=1)
#øve = pd.concat(rsidata, axis = 1)
#øve


/var/folders/br/q0k_lt357mvbfsf9815lb_yr0000gp/T/ipykernel_7366/3791602454.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closes['RSI'] = talib.RSI(closes.iloc[:, columns])
/var/folders/br/q0k_lt357mvbfsf9815lb_yr0000gp/T/ipykernel_7366/3791602454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
/var/folders/br/q0k_lt357mvbfsf9815lb_yr0000gp/T/ipykernel_7366/3791602454.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [38]:
cols = (alle.iloc[-3:] != 0).any()
websitedata = alle.iloc[-3:][cols[cols].index]
websitedata

,BTT-PERP,HNT-PERP,RUNE-PERP,SOS-PERP
,bullish_rsi_div,bearish_rsi_div,bearish_rsi_div,bullish_rsi_div
572,0,1,1,0
573,0,0,0,0
574,1,0,0,1


In [39]:
div_df = pd.DataFrame()
def get_divs(øve):
   øve = øve.iloc[-1:]
   div_df['dives'] = np.where(øve.iloc[-1:] == 0, True, False)
   return div_df
get_divs(alle)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series